In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import photogrammetry as pg

# %matplotlib qt

plt.style.use('default')

%load_ext autoreload
%autoreload 2

In [ ]:
# in case a new board type is needed
# pg.generate_charuco_board(pg.BOARD_VERT_SHAPE, pg.BOARD_SQUARE_SIZE, pg.BOARD_ARUCO_SIZE, aruco_dict=cv2.aruco.DICT_4X4_1000, gen_png=True)

In [ ]:
%autoreload 2

run = '20230622_12'
cal_path = f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/calibration/20230622_0'

extensions = ['.jpeg', '.jpg', '.JPG', '.PNG']#, '.tiff', '.TIFF']
files = []
for ext in extensions:
    files += sorted(glob.glob(os.path.join(cal_path, '**/**' + ext), recursive=True))
assert files, 'No image files found when searching for images for camera calibration.'

# check for pickled camera matrices to avoid expensive recalibration
if not (
    os.path.exists(os.path.join(cal_path, 'camera_cal_mtx.pickle')) and
    os.path.exists(os.path.join(cal_path, 'camera_cal_dist.pickle')) and
    os.path.exists(os.path.join(cal_path, 'camera_cal_optimal_camera_matrix.pickle'))
):
    # calibrate the camera for distortion
    mtx, dist, optimal_camera_matrix, roi = pg.calibrate_camera(
        cal_path,
        files,
        square_size=pg.BOARD_SQUARE_SIZE,
        aruco_size=pg.BOARD_ARUCO_SIZE,
        guess_intrinsics=True,
        plot=True,
        savefig=True
    )

    with open(os.path.join(cal_path, 'camera_cal_mtx.pickle'), 'wb') as f:
        pickle.dump(mtx, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open(os.path.join(cal_path, 'camera_cal_dist.pickle'), 'wb') as f:
        pickle.dump(dist, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open(os.path.join(cal_path, 'camera_cal_optimal_camera_matrix.pickle'), 'wb') as f:
        pickle.dump(optimal_camera_matrix, f, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(os.path.join(cal_path, 'camera_cal_mtx.pickle'), 'rb') as f:
        mtx = pickle.load(f)
    with open(os.path.join(cal_path, 'camera_cal_dist.pickle'), 'rb') as f:
        dist = pickle.load(f)
    with open(os.path.join(cal_path, 'camera_cal_optimal_camera_matrix.pickle'), 'rb') as f:
        optimal_camera_matrix = pickle.load(f)

print(mtx)
mtx_boofcv = np.array([
    [1653.5405560649892, 0, 2011.1627804232648],
    [0, 1655.631444381979,  1515.845563652173],
    [0, 0, 1]
])
print('Mine:\n', cv2.calibrationMatrixValues(mtx, (4032, 3024), 1.7e-6 * 4032, 1.7e-6 * 3024))
print('vs. BoofCV:\n', cv2.calibrationMatrixValues(mtx_boofcv, (4032, 3024), 1.7e-6 * 4032, 1.7e-6 * 3024))

print('Mine: (may not share distortion model with BoofCV)\n', dist)
dist_boofcv = np.array([
    -0.001963787455093832,
    0.0026115087601481127,
    -0.015169613897069615,
    0.02001275069414555,
    -0.01121304725629186,
    0.0023117429319429944
])
print('vs. BoofCV:\n', dist_boofcv)

`sort_images_exif ./ ./`

In [ ]:
meas_path = f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/measurement/'
files = []
for ext in extensions:
    files += sorted(glob.glob(os.path.join(meas_path, '**' + ext)))
assert files, 'No image files found when searching for images for measurement.'

In [ ]:
# use camera cal matrix to de-distort a few images to check
for file in files[:2]:
    fig, ax = plt.subplots(figsize=(12,7))
    ax.imshow(pg.load_to_gray(file, mtx, dist), cmap='bone')

In [ ]:
origin_id = 998
raft_id = 999

In [ ]:
img_data = pg.measure_images(files, mtx, dist, aruco_ids=[origin_id, raft_id], plot=False)

In [ ]:
fig, ax = plt.subplots(figsize=(12,5), subplot_kw={'projection':'3d'})
ax.view_init(elev=0, azim=-45, roll=0)

for file, entities in img_data.items():
    for id, pose in entities.items():
        for key, val in pose.items():
            print(os.path.basename(file), id, key, val)
            # if key == 'tvec_rel_camera':
            if key == 'tvec':
                if id == raft_id:
                    color = 'k'
                elif id == origin_id:
                    color = 'b'
                elif id == 'board':
                    color = 'r'
                else:
                    continue
                ax.scatter(
                    val[0],
                    val[1],
                    val[2],
                    color=color,
                    # label=id,
                    alpha=0.7
                )
ax.scatter(0,0,0, color='m', label='origin')
# ax.legend()
ax.set_box_aspect([1,1,1])
# origin = np.array([0.0, -0.00, 0.0])
# sz = .35
# ax.set_xlim(-sz + origin[0], sz + origin[0])
# ax.set_ylim(-sz + origin[1], sz + origin[1])
# ax.set_zlim(-sz/10 + origin[2], sz/10 + origin[2]);

In [ ]:
fig, ax = plt.subplots()
origin_zs = []
raft_zs = []
for file, entities in img_data.items():
    for id, pose in entities.items():
        for key, val in pose.items():
            if key == 'tvec':
                if id == origin_id:
                    origin_zs.append(val[2])
                elif id == raft_id:
                    raft_zs.append(val[2])
                else:
                    continue
origin_zs = np.array(origin_zs)
raft_zs = np.array(raft_zs)
print(origin_zs, raft_zs)
origin_inliers = np.where(np.abs(origin_zs - np.median(origin_zs)) <= np.abs(np.median(origin_zs)) + 3. * np.nanstd(origin_zs))[0]
raft_inliers = np.where(np.abs(raft_zs - np.median(raft_zs)) <= np.abs(np.median(raft_zs)) + 3. * np.nanstd(raft_zs))[0]
print(len(origin_inliers))
print(len(raft_inliers))
ax.plot(range(len(origin_zs[origin_inliers])), origin_zs[origin_inliers]*1e3, marker='o', linestyle='none')
ax.plot(range(len(raft_zs[raft_inliers])), raft_zs[raft_inliers]*1e3, marker='o', linestyle='none')
print('origin std', np.nanstd(origin_zs[origin_inliers]))
print('raft std', np.nanstd(raft_zs[raft_inliers]))
ax.set_xlabel('Scan Number')
ax.set_ylabel('z-position in board frame (mm)')

In [ ]:
thetas = []
euler_zs = []
euler_ys = []
euler_xs = []
board_tvecs_x = []
board_tvecs_y = []
board_tvecs_z = []
origin_tvecs_x = []
origin_tvecs_y = []
origin_tvecs_z = []
for file in files:
    if file in img_data.keys():
        try:
            theta = pg.estimate_angular_offset(img_data, file, 'board', raft_id)
            thetas.append(theta)
            euler_zs.append(img_data[file]['board']['euler_zyx_deg'][0])
            euler_ys.append(img_data[file]['board']['euler_zyx_deg'][1])
            euler_xs.append(img_data[file]['board']['euler_zyx_deg'][2])
            board_tvecs_x.append(img_data[file]['board']['tvec_rel_camera'][0])
            board_tvecs_y.append(img_data[file]['board']['tvec_rel_camera'][1])
            board_tvecs_z.append(img_data[file]['board']['tvec_rel_camera'][2])
            origin_tvecs_x.append(img_data[file][origin_id]['tvec_rel_camera'][0])
            origin_tvecs_y.append(img_data[file][origin_id]['tvec_rel_camera'][1])
            origin_tvecs_z.append(img_data[file][origin_id]['tvec_rel_camera'][2])
        except:
            pass
thetas = np.array(thetas)

fig, ax = plt.subplots()
ax.hist(thetas, bins=50);

import seaborn as sns

fig, ax = plt.subplots()
sns.kdeplot(euler_zs, fill=True, ax=ax)
sns.rugplot(euler_zs, ax=ax)
ax.axvline(np.mean(euler_zs), linestyle='--')
ax.set_xlabel(f'Euler z Angle (deg)\n{np.mean(euler_zs):.2f} +/- {np.std(euler_zs):.2f}')

fig, ax = plt.subplots()
sns.kdeplot(euler_ys, fill=True, ax=ax)
sns.rugplot(euler_ys, ax=ax)
ax.axvline(np.mean(euler_ys), linestyle='--')
ax.set_xlabel(f'Euler y Angle (deg)\n{np.mean(euler_ys):.2f} +/- {np.std(euler_ys):.2f}')

fig, ax = plt.subplots()
sns.kdeplot(euler_xs, fill=True, ax=ax)
sns.rugplot(euler_xs, ax=ax)
ax.axvline(np.mean(euler_xs), linestyle='--')
# ax.set_xlim(-181, -179)
ax.set_xlabel(f'Euler x Angle (deg)\n{np.mean(euler_xs):.2f} +/- {np.std(euler_xs):.2f}')

In [ ]:
# Error incurred at a given distance from the charuco board origin, given the variance in charuco board euler angles
dtheta = 0.03 * np.pi / 180.
example_dist = 0.03 * np.sqrt(2)
example_dz = example_dist * np.tan(dtheta)
print(example_dz * 1e3, 'mm')

In [ ]:
# Expected error incurred in raft position by the joint variance of board position and raft position in the camera frame
std_x = np.sqrt((np.std(origin_tvecs_x))**2 + (np.std(board_tvecs_x))**2)
std_y = np.sqrt((np.std(origin_tvecs_y))**2 + (np.std(board_tvecs_y))**2)
std_z = np.sqrt((np.std(origin_tvecs_z))**2 + (np.std(board_tvecs_z))**2)

print(std_x * 1e3, std_y * 1e3, std_z * 1e3)

In [ ]:
np.sqrt(np.std(origin_tvecs_x) ** 2 + np.std(origin_tvecs_y) ** 2 + np.std(origin_tvecs_z) ** 2)

In [ ]:
%autoreload

commanded_pts, actual_points, residuals = pg.post_process_scan(
    img_data,
    f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/results/',
    os.path.join('..', 'data', 'input', 'profiles', '25in_breadboard_raster_skipline_10x10_0.30mx0.30m.csv'),
    raft_id,
    origin_id=origin_id,
    savefig=True
)

In [ ]:
# %autoreload

# commanded_pts, actual_points, residuals = pg.post_process_repeatability(
#     img_data,
#     f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/results/',
#     os.path.join('..', 'data', 'input', 'profiles', '25in_breadboard_repeatability_N_25.csv'),
#     raft_id,
#     half_pts=True,
#     origin_id=origin_id,
#     savefig=True
# )